In [1]:
import datetime as dt
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd
import utils.utils as ut
import utils.graph_templates

from fredapi import Fred

GRAPH_NAME = "infl_expect_2"

API_KEY_PATH = ut.get_repo_root() / "fred_api_key.txt" 

fred = Fred(api_key_file = API_KEY_PATH)

pio.templates.default = 'fed_2025'

# Now is a good time to set the path to the graph output folder!
GRAPH_OUTPUT_PATH = ut.get_repo_root() / "figures"

In [31]:
breakeven = fred.get_series(
    series_id="T5YIE",
    observation_start=dt.date(2024, 1, 1),
    observation_end=dt.date.today(),
    frequency='m',
).rename("5-Year Breakeven Inflation Rate")

In [32]:
infl_df = pd.read_excel(ut.get_repo_root() / "code" / "data" / "inflation_expectations.xlsx",
                        )

infl_df = infl_df.set_index('date')

infl_df = infl_df.rename(columns = {
    'ny_fed_1': 'NY Fed Median 1-Year Ahead Expected Inflation',
    'ny_fed_5': 'NY Fed Median 5-Year Ahead Expected Inflation',
    'umich_1': 'UMich Median 1-Year Ahead Expected Inflation',
    'umich_5': 'UMich Median 5-Year Ahead Expected Inflation',
    }
)

infl_df = infl_df[['NY Fed Median 5-Year Ahead Expected Inflation', 'UMich Median 5-Year Ahead Expected Inflation']]

infl_df.index = pd.to_datetime(infl_df.index.astype(str), format="%Y%m")

infl_df = infl_df[infl_df.index >= pd.to_datetime('2024-1-1')]

infl_df = pd.concat([infl_df, breakeven], axis=1)

In [33]:
fig = go.Figure()

for col in infl_df.columns:
    fig.add_trace(
        go.Scatter(
            x=infl_df.index,
            y=infl_df[col],
            name=col,
            mode='lines',
        )
    )


# This is graph specific, but here we want the y-axis to be percent signs 
fig.update_yaxes(
    tickformat="0.2f%",
    ticksuffix="%",
    range=[0.5, 7],
)

# Again, graph specific, we have a mutliyear series and want tick marks to be years
fig.update_xaxes(
    type='date',
    tickformat='%b %Y',
    title_text=None,
    dtick="M4",
)
fig.add_hline(
    y=0,
    line_dash="solid",
    line_color="black",
)

fig.update_layout(
    barmode='relative',
    title=dict(text='Long-Term Inflation Expectations<br><sup>Monthly</br>'),
    xaxis_title=None,
    yaxis_title=None,

    legend=dict(
        orientation="h",
        x=0.2,  # Move legend inside the plot area (left)
        y=-0.1,  # Near the top
        xanchor="left",
        yanchor="top",
        font=dict(size=18)
    ),
    height=500,
    width=800,
)

fig.add_hline(
    y=2,
    line_dash="dash",
    line_color="red",
    annotation_text="Longer-Run Target",
    annotation_position="top right"
)

# This should be the same for EVERY GRAPH!
# Save it to the graph_output folder with the name matching the file, as HTML
fig.write_html(GRAPH_OUTPUT_PATH / f"{GRAPH_NAME}.html")

fig.show()
